# LIAR DETECTION GROUP PROJECT - Create LIWC ISOT file


In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from functools import reduce
import unittest
from IPython.display import display, HTML
from sklearn.utils import shuffle
# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
#assert(tf.__version__.startswith("1.8"))

import pickle
import dill
# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
from w266_common import patched_numpy_io
import timeit  #For timing


### Load ISOT data and vocabulary from pickle files  
Loading the dataset from the Information security and object technology (ISOT) Research lab at the University of Victoria School of Engineering.

The ISOT Fake News Dataset is a compilation of several thousands fake news and truthful articles, obtained from different legitimate news sites and sites flagged as unreliable by politifact.com.

In [4]:
# Read ISOT data from pickle file.
#all_data = pd.read_pickle('parsed_data/df_alldata2.pkl')  # ISOT data (CMU) tokenized and POS tags added

# NOTE: for models 2+, read in the pickle file that includes the GloVe embeddings
all_data = pd.read_pickle('parsed_data/df_alldata_embed.pkl')  # GloVe embeddings for ISOT title and text tokens

#all_data.info(memory_usage='deep', verbose=True)

In [5]:
all_data.head()

,title,text,subject,date,target,title_tokcan,title_POS,text_tokcan,text_POS,embedded_title,embedded_text
0,BRAINIAC Gets Rejected After Trying To Buy BMW...,Does anyone else out there see a future BMW ca...,Government News,"Mar 20, 2016",0,"[brainiac<allcaps>, gets, rejected, after, try...","[N, V, V, P, V, P, V, ^, P, ^, ^, ,, O, V, A, ...","[does, anyone, else, out, there, see, a, futur...","[V, N, R, P, R, V, D, A, ^, N, N, P, D, N, ,, ...","[[-0.79149, 0.86617, 0.11998, 0.00092287, 0.27...","[[0.2293, 0.34231, 0.059817, 0.083003, 0.57685..."
1,Windows 10 is Stealing Your Bandwidth (You Mig...,21st Century Wire says We ve heard a lot of no...,US_News,"April 7, 2016",0,"[windows, <number>, is, stealing, your, bandwi...","[^, $, V, V, D, N, ,, O, V, V, P, V, O, ,]","[<number>st, century, wire, says, we, ve, hear...","[A, N, ^, V, O, V, V, D, N, P, R, R, A, N, P, ...","[[0.80482, -0.39569, 1.4741, 0.39562, 0.025004...","[[-0.79149, 0.86617, 0.11998, 0.00092287, 0.27..."
2,STUNNING STORY The Media And Democrats Hid Fro...,"In an email sent on April 15, 2011, our upstan...",left-news,"Mar 2, 2017",0,"[stunning<allcaps>, story<allcaps>, the, media...","[A, N, D, N, &, N, V, P, ^, ,, R, Z, ^, ^, N, ...","[in, an, email, sent, on, april, <number>, ,, ...","[P, D, N, V, P, ^, $, ,, $, ,, D, A, N, A, ^, ...","[[-0.79149, 0.86617, 0.11998, 0.00092287, 0.27...","[[0.33042, 0.24995, -0.60874, 0.10923, 0.03637..."
3,North Korea's Kim Jong Un fetes nuclear scient...,SEOUL (Reuters) - North Korean leader Kim Jong...,worldnews,"September 10, 2017",1,"[north, korea's, kim, jong, un, fetes, nuclear...","[^, Z, ^, ^, ^, V, A, N, ,, V, N, N]","[seoul<allcaps>, (, reuters, ), -, north, kore...","[^, ,, ^, ,, ,, ^, ^, N, ^, ^, ^, V, D, A, N, ...","[[0.30059, 0.55598, -0.040589, 0.020289, -0.57...","[[-0.79149, 0.86617, 0.11998, 0.00092287, 0.27..."
4,White House developing comprehensive biosecuri...,"ASPEN, Colorado (Reuters) - The Trump administ...",politicsNews,"July 20, 2017",1,"[white, house, developing, comprehensive, bios...","[A, N, V, A, N, N, ,, A]","[aspen<allcaps>, ,, colorado, (, reuters, ), -...","[^, ,, ^, ,, ^, ,, ,, D, ^, N, V, V, D, A, A, ...","[[-0.68652, 0.80125, -0.6124, -0.1512, 0.997, ...","[[-0.79149, 0.86617, 0.11998, 0.00092287, 0.27..."


In [6]:
all_data.title[0]

'BRAINIAC Gets Rejected After Trying To Buy BMW With EBT Card…What Happens Next Is HYSTERICAL!'

In [7]:
all_data.title_tokcan[0]

['brainiac<allcaps>',
 'gets',
 'rejected',
 'after',
 'trying',
 'to',
 'buy',
 'bmw<allcaps>',
 'with',
 'ebt<allcaps>',
 'card',
 '…',
 'what',
 'happens',
 'next',
 'is',
 'hysterical<allcaps>',
 '!']

In [8]:
'''
print('length of all_data:', len(all_data))  # 44898
print('shape of all_data:', all_data.shape)  # (44898, 11)
print('length of embedded title:', len(all_data.embedded_title))  # 44898
print('shape of embedded title:', all_data.embedded_title.shape)  # (44898,)
print('dimension of single embedded title:', len(all_data.embedded_title[0]))  # 18
print('dimension of single embedded title:', len(all_data.embedded_title[1]))  # 14
print('dimension of single embedded title:', len(all_data.embedded_title[0][0]))  # 50
print('example of a title embedding:\n', all_data.embedded_title[0])
#print('dimension of single embedded text:', len(all_data.embedded_text[0][0]))  # 50
'''

"\nprint('length of all_data:', len(all_data))  # 44898\nprint('shape of all_data:', all_data.shape)  # (44898, 11)\nprint('length of embedded title:', len(all_data.embedded_title))  # 44898\nprint('shape of embedded title:', all_data.embedded_title.shape)  # (44898,)\nprint('dimension of single embedded title:', len(all_data.embedded_title[0]))  # 18\nprint('dimension of single embedded title:', len(all_data.embedded_title[1]))  # 14\nprint('dimension of single embedded title:', len(all_data.embedded_title[0][0]))  # 50\nprint('example of a title embedding:\n', all_data.embedded_title[0])\n#print('dimension of single embedded text:', len(all_data.embedded_text[0][0]))  # 50\n"

In [9]:
'''# Look at file for GloVe embedding of titles
isot_title_embed = all_data.embedded_title.values

print('isot_title_embed shape:', isot_title_embed.shape)  #(44898,)
print('isot_title_embed shape of first element:', isot_title_embed[0].shape)  # (18, 50)
print('isot_title_embed example:\n', isot_title_embed[0])
'''

"# Look at file for GloVe embedding of titles\nisot_title_embed = all_data.embedded_title.values\n\nprint('isot_title_embed shape:', isot_title_embed.shape)  #(44898,)\nprint('isot_title_embed shape of first element:', isot_title_embed[0].shape)  # (18, 50)\nprint('isot_title_embed example:\n', isot_title_embed[0])\n"

In [10]:
'''
test = np.reshape(isot_title_embed, (-1, 50)) 
print(text.shape)
'''

'\ntest = np.reshape(isot_title_embed, (-1, 50)) \nprint(text.shape)\n'

In [11]:
# Read ISOT vocab from pickle file.

vocab = pd.read_pickle('parsed_data/vocab.pkl')  # ISOT data (CMU) tokenized and POS tags added

In [12]:
print("{:,} words".format(vocab.size))  # Note: this combines words from ISOT "title" AND "text" fields!
print("wordset: ",vocab.ordered_words()[:30])
print(vocab)

152,182 words
wordset:  ['<s>', '</s>', '<unk>', 'the', ',', '.', 'to', 'of', 'a', 'and', 'in', 'that', 'on', '<number>', 'for', 's', 'is', 'he', 'said', 'trump', 'it', 'with', 'was', 'as', 'his', 'by', 'has', 'be', 'have', 'not']


In [13]:
print('ISOT ALL target=real:', len(all_data.target[all_data.target == '1']))
print('ISOT ALL target=fake:', len(all_data.target[all_data.target == '0']))

ISOT ALL target=real: 21417
ISOT ALL target=fake: 23481


### Load ISOT LIWC features

In [14]:
liwc_isot_orig = pd.read_csv('datasets/ISOT_FakeNews/LIWC2015 Results (all words).csv')
liwc_isot_orig.head()

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,19,36,39,47,55,56,57,71,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,donald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,trump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,out,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,embarrassing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
liwc_isot_clean = liwc_isot_orig.fillna(0)
liwc_isot_clean.head()

,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,donald,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,trump,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,sends,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,out,X,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,embarrassing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [16]:
liwc_isot_clean = liwc_isot_clean.replace('X','1')
liwc_isot_clean.head()

,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,donald,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,trump,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,sends,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,out,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,embarrassing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [17]:
#liwc_isot_clean.loc[3]

In [18]:
liwc_isot_clean.values

array([['donald', 0, 0, ..., 0, 0, 0.0],
       ['trump', 0, 0, ..., 0, 0, 0.0],
       ['sends', 0, 0, ..., 0, 0, 0.0],
       ...,
       ['lukita', 0, 0, ..., 0, 0, 0.0],
       ['perusahaan', 0, 0, ..., 0, 0, 0.0],
       ['perdangangan', 0, 0, ..., 0, 0, 0.0]], dtype=object)

In [19]:
# example: http://ai.intelligentonlinetools.com/ml/convert-word-to-vector-glove-python/

#file = 'datasets/ISOT_FakeNews/LIWC2015 Results (all words).csv'
#test = pd.read_table(file, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [17]:
liwc_isot_clean['id'] = vocab.words_to_ids(liwc_isot_clean['Word'])
liwc_isot_clean.head()

,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74,id
0,donald,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,81
1,trump,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,19
2,sends,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,5026
3,out,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,65
4,embarrassing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2787


In [18]:
liwc_isot_clean['id'].min()

2

In [19]:
liwc_isot_clean['id'].max()

152181

In [20]:
print(len(liwc_isot_clean)) # ~ 26k unknown values???  They appear to be assigned to id=2.

126466


In [21]:
cols = liwc_isot_clean.columns.tolist()
print(cols)

['Word', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler', 'Unnamed: 74', 'id']


In [22]:
newcols = ['Word', 'id', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler', 'Unnamed: 74']

In [23]:
# reorder columns
liwc_isot_clean = liwc_isot_clean[newcols]
liwc_isot_clean.head()

,Word,id,function,pronoun,ppron,i,we,you,shehe,they,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,donald,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,trump,19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,sends,5026,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,out,65,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,embarrassing,2787,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [24]:
# rename ids to id
#liwc_isot_clean = liwc_isot_clean.rename(index=str, columns={"ids": "id"})
#liwc_isot_clean.head()

In [27]:
#print(liwc_isot_clean.iloc[0:100])
#print(liwc_isot_clean[liwc_isot_clean.id==1])
print(liwc_isot_clean[liwc_isot_clean.id==2])  # this appears to be the assignment for unknown tokens
print(len(liwc_isot_clean[liwc_isot_clean.id==2]))  # 36426

                  Word  id function pronoun ppron  i we you shehe they  \
67                2018   2        0       0     0  0  0   0     0    0   
76                  31   2        0       0     0  0  0   0     0    0   
77           2017trump   2        0       0     0  0  0   0     0    0   
83               welll   2        0       0     0  0  0   0     0    0   
88             kind of   2        0       0     0  0  0   0     0    0   
111        talbertswan   2        0       0     0  0  0   0     0    0   
112             2017no   2        0       0     0  0  0   0     0    0   
115      calvinstowell   2        0       0     0  0  0   0     0    0   
116           2017your   2        0       0     0  0  0   0     0    0   
129       mirandayaver   2        0       0     0  0  0   0     0    0   
130             2017do   2        0       0     0  0  0   0     0    0   
146     alansandoval13   2        0       0     0  0  0   0     0    0   
147            2017who   2        0   

In [25]:
print(liwc_isot_clean[liwc_isot_clean.id==3])

   Word  id function pronoun ppron  i we you shehe they     ...     money  \
35  the   3        1       0     0  0  0   0     0    0     ...         0   

   relig death informal swear netspeak assent nonflu filler Unnamed: 74  
35     0     0        0     0        0      0      0      0         0.0  

[1 rows x 76 columns]


In [30]:
# Since there are so many duplicates for id=2, us groupby function to collapse them.

In [28]:
liwc_isot_clean2 = liwc_isot_clean.groupby(['id']).first()
liwc_isot_clean2.head()
print(len(liwc_isot_clean2))  # 90041

90041


In [30]:
126466 - 90041  # 36425 -> Essentially all dups came from ID-2.  However, there appear to be 25715 missing 
#   word id's in the LIWC set relative to the ISOT vocabulary.

36425

In [31]:
152181 - 126466  # 25715 = difference between vocabulary size and words in LIWC set

25715

In [32]:
liwc_isot_clean2.head()

,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
id,,,,,,,,,,,,,,,,,,,,,
2,2018,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,the,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,to,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
7,of,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,a,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [38]:
#print(liwc_isot_clean2.loc[1])  ## KeyError: 'the label [1] is not in the [index]'
#print(liwc_isot_clean2.loc[2])

### NOTE: we don't have continuous index (id) values at this point, so tf.embedding_lookup won't work; need to remedy....

In [39]:
liwc_isot_clean2['id'] = liwc_isot_clean2.index
liwc_isot_clean2.head()

,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74,id
id,,,,,,,,,,,,,,,,,,,,,
2,2018,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2
3,the,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3
6,to,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,6
7,of,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,7
8,a,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,8


In [40]:
newcols = ['id', 'Word', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler', 'Unnamed: 74']
#len(newcols)  # 76

# reorder columns
liwc_isot_clean2 = liwc_isot_clean2[newcols]
liwc_isot_clean2.head()

,id,Word,function,pronoun,ppron,i,we,you,shehe,they,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
id,,,,,,,,,,,,,,,,,,,,,
2,2,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,3,the,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,6,to,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
7,7,of,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,8,a,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [46]:
print(liwc_isot_clean2[liwc_isot_clean2.id==1].isnull())

Empty DataFrame
Columns: [id, Word, function, pronoun, ppron, i, we, you, shehe, they, ipron, article, prep, auxverb, adverb, conj, negate, verb, adj, compare, interrog, number, quant, affect, posemo, negemo, anx, anger, sad, social, family, friend, female, male, cogproc, insight, cause, discrep, tentat, certain, differ, percept, see, hear, feel, bio, body, health, sexual, ingest, drives, affiliation, achieve, power, reward, risk, focuspast, focuspresent, focusfuture, relativ, motion, space, time, work, leisure, home, money, relig, death, informal, swear, netspeak, assent, nonflu, filler, Unnamed: 74]
Index: []

[0 rows x 76 columns]


In [50]:
#print(liwc_isot_clean2.loc[3].isnull())
#print(liwc_isot_clean2.loc[4].isnull())
#print(liwc_isot_clean2[liwc_isot_clean2.index==2])

In [47]:
print(liwc_isot_clean2.index.max)

<bound method IndexOpsMixin.max of Int64Index([     2,      3,      6,      7,      8,      9,     10,     11,
                12,     14,
            ...
            152151, 152157, 152158, 152161, 152163, 152164, 152169, 152170,
            152174, 152181],
           dtype='int64', name='id', length=90041)>


In [48]:
max_index = 152181
liwc_isot_clean3 = liwc_isot_clean2

In [ ]:
## EXAMPLE:

#years = df['year'].unique()
#missing_years = [y for y in range(min(years), max(years)+1) if y not in years]


In [49]:
## APPLY HERE
ids_present = liwc_isot_clean2.index.unique()
print(ids_present)

Int64Index([     2,      3,      6,      7,      8,      9,     10,     11,
                12,     14,
            ...
            152151, 152157, 152158, 152161, 152163, 152164, 152169, 152170,
            152174, 152181],
           dtype='int64', name='id', length=90041)


In [59]:
ids_present2 = list(ids_present)
print(ids_present2)

[2, 3, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 23

In [50]:
ids_missing = [i for i in range(0,152181) if i not in ids_present]
print(ids_missing)

[0, 1, 4, 5, 13, 32, 33, 35, 42, 44, 70, 78, 82, 114, 149, 172, 209, 221, 264, 269, 286, 296, 322, 324, 387, 392, 414, 455, 498, 517, 518, 531, 545, 546, 573, 590, 591, 710, 734, 740, 743, 748, 773, 870, 903, 907, 910, 925, 957, 1019, 1034, 1047, 1056, 1094, 1127, 1200, 1202, 1214, 1371, 1381, 1383, 1427, 1431, 1452, 1455, 1463, 1464, 1492, 1495, 1506, 1544, 1554, 1560, 1563, 1569, 1604, 1607, 1636, 1656, 1671, 1714, 1736, 1762, 1795, 1810, 1815, 1852, 1879, 1922, 1956, 1970, 1974, 1986, 1997, 2087, 2097, 2132, 2138, 2165, 2230, 2242, 2297, 2304, 2332, 2384, 2425, 2426, 2447, 2486, 2492, 2564, 2575, 2586, 2609, 2634, 2653, 2654, 2670, 2680, 2688, 2691, 2720, 2763, 2767, 2782, 2799, 2826, 2854, 2855, 2859, 2862, 2896, 2900, 2922, 2954, 3012, 3050, 3066, 3074, 3097, 3105, 3112, 3145, 3154, 3170, 3198, 3199, 3221, 3284, 3305, 3306, 3313, 3319, 3323, 3361, 3383, 3388, 3406, 3425, 3431, 3498, 3503, 3543, 3565, 3577, 3610, 3611, 3617, 3637, 3648, 3655, 3668, 3679, 3709, 3735, 3759, 3763, 380

#### Create new full dataframe having all rows (0 - 152,181) and initialize to zeros.  Then, take the rows in the liwc_isot_clean2 matrix and overwrite the corresponding rows in the full dataframe.

In [54]:
fullmat = np.zeros(152182*76).reshape(152182,76)
print(fullmat)
print(fullmat.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(152182, 76)


In [63]:
#newcols2 = ['function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler', 'Unnamed: 74']

liwc_isot = pd.DataFrame(fullmat, columns = newcols)
liwc_isot.head()

,id,Word,function,pronoun,ppron,i,we,you,shehe,they,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# copy rows with ids present to full dataframe.  Use UPDATE method

liwc_isot.update(liwc_isot_clean2)


In [66]:
print(len(liwc_isot))
liwc_isot.head(10)

152182


,id,Word,function,pronoun,ppron,i,we,you,shehe,they,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,2.0,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,3.0,the,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
5,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,6.0,to,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
7,7.0,of,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,8.0,a,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
9,9.0,and,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [67]:
liwc_isot2 = liwc_isot
liwc_isot2.drop(['id', 'Word'], axis=1, inplace=True)
liwc_isot2.head()

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [68]:
liwc_isot2 = liwc_isot2.astype(int)
liwc_isot2.head()

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
#liwc_isot2.head(50)

In [71]:
# convert to np array
liwc_isot2_array = liwc_isot2.values

In [72]:
print(liwc_isot2_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [74]:
# Pickle the LIWC ISOT df data
liwc_isot2.to_pickle('parsed_data/liwc_isot2.pkl')

## STOP HERE.....

In [29]:
# Make the id column the df index
# liwc_isot = liwc_isot_clean2.set_index('id', inplace=False)

In [33]:
# Sort based on the index 
#liwc_isot2 = liwc_isot.sort_index()
#liwc_isot2.head()


,Word,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Unnamed: 74
2,swigging,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,southernbaptist,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,conzelmann,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,1547,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,prayfortexas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
